In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [2]:
with open('/Users/Lindsey/.secret/yelp_api.json') as f: 
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [3]:
LOCATION = "Boston, MA, 02134"
TERM = "Lobster Roll"

In [4]:
JSON_FILE = f"Data/results_in_progress_Lobster_Roll.json"
JSON_FILE



'Data/results_in_progress_Lobster_Roll.json'

In [5]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
  
    if len(folder)>0:
       
        os.makedirs(folder,exist_ok=True)
        
        
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
else:
    print(f"[i] {JSON_FILE} already exists.")



[i] Data/results_in_progress_Lobster_Roll.json not found. Saving empty list to file.


In [6]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [7]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
total_results = results['total']
total_results

202

In [9]:
results_per_page = len(results['businesses'])
results_per_page



20

In [10]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

11

In [11]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
for i in tqdm_notebook(range(n_pages)):
    time.sleep(.2) 

  0%|          | 0/11 [00:00<?, ?it/s]

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/11 [00:00<?, ?it/s]

In [14]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,T5_3LoIuOnilA7Z5sdfGJA,alive-and-kicking-lobsters-cambridge,Alive & Kicking Lobsters,https://s3-media3.fl.yelpcdn.com/bphoto/wiNauY...,False,https://www.yelp.com/biz/alive-and-kicking-lob...,913,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 42.362678527832, 'longitude': -71...",[delivery],$$,"{'address1': '269 Putnam Ave', 'address2': '',...",+16178760451,(617) 876-0451,1170.763247
1,op2-8aK7VaNvXG8Yxas35g,the-hourly-oyster-house-cambridge,The Hourly Oyster House,https://s3-media2.fl.yelpcdn.com/bphoto/1A9Ctb...,False,https://www.yelp.com/biz/the-hourly-oyster-hou...,434,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 42.37291, 'longitude': -71.11904}","[delivery, pickup]",$$$,"{'address1': '15 Dunster St', 'address2': None...",+16177652342,(617) 765-2342,1631.375282
2,HUtHbHjyu114ptcilT_M6g,the-daily-catch-brookline,The Daily Catch,https://s3-media3.fl.yelpcdn.com/bphoto/1hzWA0...,False,https://www.yelp.com/biz/the-daily-catch-brook...,438,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.0,"{'latitude': 42.34611, 'longitude': -71.12766}","[delivery, pickup]",$$,"{'address1': '441 Harvard St', 'address2': '',...",+16177342700,(617) 734-2700,1471.971377
3,i6uHrxuS1D_6V8WhaWLy5A,lukes-lobster-back-bay-boston,Luke's Lobster Back Bay,https://s3-media1.fl.yelpcdn.com/bphoto/EnMeIE...,False,https://www.yelp.com/biz/lukes-lobster-back-ba...,1925,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 42.34869123638961, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '75 Exeter Street', 'address2': '...",+18573504626,(857) 350-4626,4132.803723
4,JZUTavw4SsZ-iiExULq1_g,lobstah-on-a-roll-boston-2,Lobstah On A Roll,https://s3-media2.fl.yelpcdn.com/bphoto/ZdYDW-...,False,https://www.yelp.com/biz/lobstah-on-a-roll-bos...,115,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 42.34928, 'longitude': -71.08272}","[delivery, pickup]",$$,"{'address1': '254 Newbury St', 'address2': Non...",+16174312284,(617) 431-2284,3839.367035


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
197,JzUlNRnJ2d10dZgKYHzdeg,seiyo-boston,Seiyo,https://s3-media3.fl.yelpcdn.com/bphoto/5Mz_6b...,False,https://www.yelp.com/biz/seiyo-boston?adjust_c...,243,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 42.337133, 'longitude': -71.076848}","[pickup, delivery]",$$,"{'address1': '1721 Washington St', 'address2':...",+16174472183,(617) 447-2183,4837.197671
198,XgJWwSXomuSblxFBE-AxpA,petit-robert-bistro-boston-3,Petit Robert Bistro,https://s3-media1.fl.yelpcdn.com/bphoto/aWNVmS...,False,https://www.yelp.com/biz/petit-robert-bistro-b...,490,"[{'alias': 'french', 'title': 'French'}, {'ali...",3.5,"{'latitude': 42.34283, 'longitude': -71.07858}","[pickup, delivery]",$$,"{'address1': '480 Columbus Ave', 'address2': '...",+16178670600,(617) 867-0600,4414.367462
199,GwUD1wRUcKv__r2bv_CpJw,the-westin-copley-place-boston-boston-3,"The Westin Copley Place, Boston",https://s3-media2.fl.yelpcdn.com/bphoto/KTRAis...,False,https://www.yelp.com/biz/the-westin-copley-pla...,413,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.5,"{'latitude': 42.348324, 'longitude': -71.077464}",[],$$$,"{'address1': '10 Huntington Ave', 'address2': ...",+16172629600,(617) 262-9600,4283.300841
200,Sevr2MzD_5cQbIXI-HkM8Q,sheraton-boston-hotel-boston,Sheraton Boston Hotel,https://s3-media1.fl.yelpcdn.com/bphoto/oa90Pg...,False,https://www.yelp.com/biz/sheraton-boston-hotel...,507,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali...",3.0,"{'latitude': 42.3462530906186, 'longitude': -7...",[],$$$,"{'address1': '39 Dalton Street', 'address2': '...",+16172362000,(617) 236-2000,3825.287685
201,xLezzT2Xn2vTCNMbVrf7zw,typhoon-asian-bistro-boston,Typhoon Asian Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/ib1zXs...,False,https://www.yelp.com/biz/typhoon-asian-bistro-...,294,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.0,"{'latitude': 42.3496, 'longitude': -71.08016}","[delivery, pickup]",$$,"{'address1': '725 Boylston St', 'address2': ''...",+16178598181,(617) 859-8181,4031.310040


In [15]:
final_df.duplicated(subset='id').sum()



1

In [16]:
final_df.drop_duplicates(subset = "id", inplace = True)

In [17]:
final_df.duplicated(subset='id').sum()


0

In [18]:
final_df.to_csv('Data/final_results_Lobster_Roll.csv.gz', compression='gzip',index=False)

